In [1]:
# Here we take care of paths.

from pathlib import Path
import os
print('Starting path:' + os.getcwd())
if os.getcwd()[-16:] == 'image-to-tex-OCR':
    pass
else:
    PATH = Path().resolve().parents[0]
    os.chdir(PATH)

# make sure you are in Paragraph_to_Tex folder
print('Current path:' + os.getcwd())

Starting path:/Users/gregory/PROJECT_ML/PROJECT_AMINE/image-to-tex-OCR/Jupyter_Notebooks
Current path:/Users/gregory/PROJECT_ML/PROJECT_AMINE/image-to-tex-OCR


In [2]:
from Data.Data_Module import Data_Module
from Models.Printed_Tex_Transformer import ResNetTransformer
from Lightning_Models.Printed_Tex_Lit_Model import LitResNetTransformer
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from PIL import Image
import torch
from pytorch_lightning.loggers import TensorBoardLogger
from torchvision import transforms
from PIL import Image
from IPython.display import display, Math

dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Loading Vocabulary

**Note: One needs to use the vocabulary json of the trained model 
and our paths come with corresponding vocabulary**

 - Path files and Vocabulary json are located in `Models_Parameters_Log` folder
 
- set the `load_vocabulary = True` parameter of Data_Module below

- set the `vocabulary_path = Path_to_json` parameter of Data_Module below

- make sure the vocabulary size mathches the original trained vocabulary size of 550 (if you are using our weights)



N.B. this needs to be improved

In [3]:
# Generate Data Module by uploading images and formulas
# images need to be in the folder Data/Data_Bank/generated_png_images
# formulas need to be in Data/Data_Bank/final_png_formulas.txt
# image filenames need to be in Data/Data_Bank/corresponding_png_images.txt

dataset = Data_Module(stage = 'fit',
                 set_max_label_length = 256,
                 number_png_images_to_use_in_dataset=188021,
                 labels_transform='default',
                image_transform_name ='alb',
                 train_val_fraction = 0.99,
                   
                load_vocabulary = True,
                vocabulary_path = 'Models_Parameters_Log/258_Test_run.json',


                 batch_size = 128,
                num_workers = 8,
                data_on_gpu = True,
                )


# Generates DataFrame and vocabulary, tokezniers etc
#dataset.prepare_data()

# prepeares the dataloaders with transformations and splits train/val
# dataset.setup(stage = 'fit')

AttributeError: 'dict' object has no attribute 'iteritems'

In [ ]:
print( 
    'Vocabulary size:',len(dataset.vocabulary),
    '\nMax label length:', dataset.max_label_length,
    "\nStart <S> goes to index ",dataset.vocabulary['<S>'],
      "\nEnd <E> goes to index ",dataset.vocabulary['<E>'],
      "\nPadding <P> goes to index ",dataset.vocabulary['<P>'],
)

In [ ]:
# Initiate the model
model = ResNetTransformer(dataset=dataset).to(dev)

In [ ]:
# Load the weights

model.load_state_dict(torch.load(("Models_Parameters_Log/258_Test_run.pth"), map_location=torch.device('cpu')))


### Inference Tests on various images


In [ ]:
# Helper Function to convert prediction labels to strings
def token_to_strings(tokens):
    mapping = dataset.vocabulary
    inverse_mapping =dataset.inverse_vocabulary
    s=''
    if tokens.shape[0] ==1:
        tokens = tokens[0]
    for number in tokens:
        
        letter = inverse_mapping[number.item()]
        s= s +" " + str(letter)
    return s

In [ ]:
### ALB
my_image_path= "Jupyter_Notebooks/test_photos/my_image.png"
my_image = Image.open(my_image_path).convert("L") 
my_image_tensor = dataset.image_transform_test(image=np.array(my_image))['image']
print(display(my_image))
print('\nPredicted formula:')
with torch.no_grad():
    model.eval()
    my_prediction =  model.predict(my_image_tensor.unsqueeze(0).to(dev))
print(token_to_strings(my_prediction))
print(display(Math(token_to_strings(my_prediction))))


In [ ]:
dataset.inverse_vocabulary